In [6]:
import json
from typing import Dict, Any
from pathlib import Path
from ocr import ImageExtractor

template_name: str = "lloyds"
identifier: str = "september"

# Use Path to handle file paths more elegantly
root_directory = Path().resolve()

parent_directory = root_directory.parent
grandparent_directory = parent_directory.parent

pdf_path = grandparent_directory / "data" / "bank_statements" / f"{template_name}" / "pdf" / f"{template_name}_{identifier}.pdf"
template_path: Path = parent_directory / "templates" / f"{template_name}_template.json"
pdf_data_path: Path = parent_directory / "pdf_data" / f"{template_name}_{identifier}_pdf_data.json"

# Load JSON data using context manager for better file handling
with open(template_path) as template_file:
    template: Dict[str, Any] = json.load(template_file)

with open(pdf_data_path) as pdf_data_file:
    pdf_data: Dict[str, Any] = json.load(pdf_data_file)

output_data_path: Path = root_directory / "src" / "outputs" / f"{template_name}_{identifier}_output.json"
from pdf_utils import ImageDrawer

jpg_image = ImageDrawer.create_jpg_image(pdf_path, 1)

from io import BytesIO

# Convert the JPG image to bytes
jpg_image_bytes = BytesIO()
jpg_image.save(jpg_image_bytes, format='JPEG')
jpg_image_bytes.seek(0)

# Create an instance of ImageExtractor with the image bytes and coordinates
coordinates = {'top_left': {'x': 0.197, 'y': 0.605998}, 'bottom_right': {'x': 0.441, 'y': 0.635689}}

image_extractor = ImageExtractor(jpg_image_bytes.getvalue(), coordinates)
extracted_text = image_extractor.extract_text()

# Optionally, print the extracted text
print(extracted_text)

def extract_section(jpg_bytes, coordinates):
    from PIL import Image
    import io

    # Load the image from the byte data
    image = Image.open(io.BytesIO(jpg_bytes))

    # Convert decimal coordinates to pixel values
    width, height = image.size
    top_left_x = int(coordinates["top_left"]["x"] * width)
    top_left_y = int(coordinates["top_left"]["y"] * height)
    bottom_right_x = int(coordinates["bottom_right"]["x"] * width)
    bottom_right_y = int(coordinates["bottom_right"]["y"] * height)

    # Crop the image to the specified coordinates
    section_image = image.crop((top_left_x, top_left_y, bottom_right_x, bottom_right_y))
    
    return section_image


section_image = extract_section(jpg_image_bytes.getvalue(), coordinates)
section_image.show()


AIRBNB * HM8Y5A4RR

